In [1]:
import pandas as pd
import numpy as np

In [2]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
from gensim.models import Word2Vec, KeyedVectors

In [43]:
from string import punctuation

In [51]:
from nltk.tokenize import word_tokenize

In [6]:
modelo = KeyedVectors.load_word2vec_format("../../../dados/skip_s100/skip_s100.txt")

In [45]:
# Create a translation table to replace vowels with numbers and delete spaces
translation_table = str.maketrans("", "", punctuation)

hello world


In [49]:
df = pd.read_csv("../../../dados/nlp/imdb-reviews-pt-br.csv", encoding="latin1")
df["text_final"] = df["text_pt"].replace({np.nan: ""}).str.lower().str.translate(translation_table)
df.head()

,id,text_en,text_pt,sentiment,text_final
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,mais uma vez o sr costner arrumou um filme por...
1,2,This is an example of why the majority of acti...,Este Ã© um exemplo do motivo pelo qual a maior...,neg,este ã© um exemplo do motivo pelo qual a maior...
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg,primeiro de tudo eu odeio esses raps imbecis q...
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever mÃºsicas...,neg,nem mesmo os beatles puderam escrever mãºsicas...
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latÃ£o nÃ£o Ã© uma palavra ...,neg,filmes de fotos de latã£o nã£o ã© uma palavra ...


In [53]:
def tokenizar( texto ):
    return word_tokenize(texto, language='portuguese')

In [57]:
df["texto_final"] = df["text_final"].apply( tokenizar )

In [59]:
df.head()

,id,text_en,text_pt,sentiment,text_final,texto_final
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,mais uma vez o sr costner arrumou um filme por...,"[mais, uma, vez, o, sr, costner, arrumou, um, ..."
1,2,This is an example of why the majority of acti...,Este Ã© um exemplo do motivo pelo qual a maior...,neg,este ã© um exemplo do motivo pelo qual a maior...,"[este, ã©, um, exemplo, do, motivo, pelo, qual..."
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg,primeiro de tudo eu odeio esses raps imbecis q...,"[primeiro, de, tudo, eu, odeio, esses, raps, i..."
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever mÃºsicas...,neg,nem mesmo os beatles puderam escrever mãºsicas...,"[nem, mesmo, os, beatles, puderam, escrever, m..."
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latÃ£o nÃ£o Ã© uma palavra ...,neg,filmes de fotos de latã£o nã£o ã© uma palavra ...,"[filmes, de, fotos, de, latã£o, nã£o, ã©, uma,..."


In [61]:
maior_frase = 0
for linha in df["texto_final"]:
    if len(linha) > maior_frase:
        maior_frase = len(linha)
maior_frase

953

In [10]:
df["output"] = df["sentiment"].replace({"neutral": 0, "negative": 0, "positive": 1})

C:\Users\anton\AppData\Local\Temp\ipykernel_19568\1959351941.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["output"] = df["sentiment"].replace({"neutral": 0, "negative": 0, "positive": 1})


In [14]:
vetorizador = CountVectorizer(max_features=1000, binary=True, lowercase=True)
# deixar 5 
# raiva 1
# [0, 0, 0, 0, 0, 0, 1]      raiva  - 1
# [0, 0, 1, 0, 0, 0, 0]      deixar - 5 

In [15]:
bow = vetorizador.fit_transform( df["text"] )


In [16]:
vocabulario = vetorizador.get_feature_names_out()

In [17]:
features = len(vocabulario)
features

1000

In [18]:
all_X = pd.DataFrame.sparse.from_spmatrix( bow, columns=vocabulario ).astype(np.float32)

In [19]:
all_X

,10,100,11,12,15,1st,20,24,2day,30,...,yet,you,your,yours,yourself,youtube,yum,yummy,yup,â½
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32291,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32293,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [117]:
train_X = all_X[:df_train.shape[0]]
test_X = all_X[df_train.shape[0]:]

In [119]:
train_Y = df["output"][:df_train.shape[0]]
test_Y = df["output"][df_train.shape[0]:]

In [127]:
modelo = Sequential()
modelo.add( Input( (features, ) ) )
modelo.add( Dense( 128, activation="relu" ) )
modelo.add( Dense( 16, activation="relu" ) )
modelo.add( Dense( 1, activation="sigmoid" ) )
modelo.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │         128,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 16)                  │           2,064 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 130,209 (508.63 KB)

 Trainable params: 130,209 (508.63 KB)

 Non-trainable params: 0 (0.00 B)

In [129]:
modelo.compile(loss="binary_crossentropy", optimizer="RMSprop", metrics=["accuracy"])

In [131]:
historico = modelo.fit(train_X, train_Y, epochs=10, batch_size=32)

Epoch 1/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7755 - loss: 0.4963
Epoch 2/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8554 - loss: 0.3638
Epoch 3/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8710 - loss: 0.3352
Epoch 4/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8772 - loss: 0.3200
Epoch 5/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8847 - loss: 0.3036
Epoch 6/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8925 - loss: 0.2894
Epoch 7/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8981 - loss: 0.2769
Epoch 8/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9020 - loss: 0.2670
Epoch 9/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9097 - loss: 0.2560
Epoch 10/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9135 - loss: 0.2427


In [133]:
modelo.evaluate( test_X, test_Y )

151/151 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8103 - loss: nan


[nan, 0.6191069483757019]

In [139]:
predict_vectorizer = CountVectorizer(max_features=features, binary=True, 
                                     lowercase = True, vocabulary=vocabulario)

In [155]:
texto = "I love traffic"
bow_predict = predict_vectorizer.fit_transform([texto])
predict_X = pd.DataFrame.sparse.from_spmatrix( bow_predict, columns=vocabulario ).astype(np.float32)

In [157]:
"negativo" if modelo.predict( predict_X )[0][0] < 0.5 else "positivo"

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step


'positivo'